In [1]:
from InSituToolkit.imaging_database import write_experiment
import imaging_db.database.db_operations as db_ops
import imaging_db.filestorage.s3_storage as s3_storage
import imaging_db.filestorage.local_storage as local_storage
import imaging_db.utils.db_utils as db_utils
import os

# information taken from InSituToolkit notebooks, which happened to use the LungInsitu dataset
# we must pull all image id's that contain "SDG" and make an experiment for them.
# We assume that each file is a single round of imaging

spot_channels = ['Opal_570_low', 'Opal_620', 'Opal_690']
nuc_channel = ['DAPI']
db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'


In [12]:
# Make one example experiment

example_dataset = 'SDG-2019-10-07-22-22-22-0098'
output_dir = "/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/temp/" + example_dataset

try:
    os.mkdir(output_dir)
except OSError:
    print ("Creation of the directory %s failed" % output_dir)
else:
    print ("Successfully created the directory %s " % output_dir)

write_experiment(db_credentials, output_dir, example_dataset,
                 spot_channels=spot_channels, nuc_channels=nuc_channel,
                 positions=[12]
                )

Successfully created the directory /Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/temp/SDG-2019-10-07-22-22-22-0098 


TypeError: cannot unpack non-iterable NoneType object

In [8]:
# accessing imagingDB and pulling the dataset id's. What we want is a set of unique id's that we then
# write into experiment files

def getIDs(db_credentials, string):
    credentials_str = db_utils.get_connection_str(db_credentials)
    with db_ops.session_scope(credentials_str) as session:
        frames = session.query(db_ops.DataSet) 

    set_of_ids = list()
    for f in frames:
        name = f.dataset_serial
        if string in name:
            set_of_ids.append(name)
    return list(set_of_ids)
        
def getPositions(db_credentials, dataset_identifier):
    credentials_str = db_utils.get_connection_str(db_credentials)
    
    with db_ops.session_scope(credentials_str) as session:
        frames = session.query(db_ops.Frames) \
            .join(db_ops.FramesGlobal) \
            .join(db_ops.DataSet)  \
            .filter(db_ops.DataSet.dataset_serial == dataset_identifier) 
    positions = set()
    for f in frames:
        positions.add(f.pos_idx)
    return list(positions)

In [9]:
list_of_datasets = getIDs(db_credentials, 'SDG')

In [10]:
list_of_datasets

['SDG-2019-10-08-13-30-00-0001',
 'SDG-2019-10-07-13-30-00-0001',
 'SDG-2019-10-07-13-30-00-0002',
 'SDG-2019-10-07-13-30-00-0003',
 'SDG-2019-10-07-13-30-00-0099',
 'SDG-2019-10-07-13-30-00-0098',
 'SDG-2019-10-07-22-22-22-0098',
 'SDG-2019-10-28-16-53-00-0001',
 'SDG-2019-10-28-16-53-00-0002',
 'SDG-2019-10-28-16-53-00-0003',
 'SDG-2019-10-24-15-00-00-0001',
 'SDG-2019-10-16-14-30-00-0001',
 'SDG-2019-10-16-14-30-00-0002',
 'SDG-2019-10-16-12-40-00-0001',
 'SDG-2019-10-16-12-40-00-0002',
 'SDG-2019-10-16-11-09-00-0001',
 'SDG-2019-10-16-10-49-00-0001',
 'SDG-2019-10-16-10-37-00-0001',
 'SDG-2019-10-16-09-21-00-0001',
 'SDG-2019-10-16-09-21-00-0002',
 'SDG-2019-10-16-09-21-00-0003',
 'SDG-2019-10-15-15-10-00-0001',
 'SDG-2019-10-15-15-10-00-0002',
 'SDG-2019-10-15-15-10-00-0003',
 'SDG-2019-10-08-16-22-00-0001',
 'SDG-2019-10-08-16-22-00-0002',
 'SDG-2019-10-08-16-22-00-0003',
 'SDG-2019-10-08-16-00-00-0001',
 'SDG-2019-10-08-15-10-00-0001',
 'SDG-2019-10-08-15-10-00-0002',
 'SDG-2019

In [5]:
# Construct experiment files for each dataset, finding the positions as well

list_of_datasets = getIDs(db_credentials, 'SDG')

for dataset_identifier in list_of_datasets[0:1]:
    pos = getPositions(db_credentials, dataset_identifier)
    
    # Saving experiment file to the repo
    output_dir = "/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/" + dataset_identifier + "/"

    # first try to make the directory
    try:
        os.mkdir(output_dir)
    except OSError:
        print ("Creation of the directory %s failed" % output_dir)
    else:
        print ("Successfully created the directory %s " % output_dir)

    #write the experiment
    write_experiment(db_credentials, output_dir, dataset_identifier,
                 spot_channels=spot_channels, nuc_channels=nuc_channel,
                 positions=pos
                )

Successfully created the directory /Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/SDG-2019-10-08-15-10-00-0002/ 


TypeError: cannot unpack non-iterable NoneType object